In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize

from LocalitySensitiveHashing import *

#from annoy import AnnoyIndex

In [2]:
!pip install LocalitySensitiveHashing
!pip install BitVector

     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for LocalitySensitiveHashing: filename=LocalitySensitiveHashing-1.0.1-py3-none-any.whl size=21905 sha256=4b6c7bf9d3c565b0d4600fcc673d5b42c37be81527e6961fbd0fc07530a10294
  Stored in directory: /root/.cache/pip/wheels/ce/d1/63/c4289a64f47cb9615af549422ea9c6a5e681561b24422892ef
Successfully built LocalitySensitiveHashing
     |████████████████████████████████| 161 kB 10.1 MB/s 
  Created wheel for BitVector: filename=BitVector-3.5.0-py3-none-any.whl size=43508 sha256=fb669aeb19ab32ee6191fd91f8eb201d08a28045694fd1304c4feadb7ef1f516
  Stored in directory: /root/.cache/pip/wheels/6f/6c/b6/c8f6bb33d1e9638df10214d2b08cb70647936dbf4595fabbc3
Successfully built BitVector


In [32]:
train_data = pd.read_csv("ramen-ratings.csv")
train_data = train_data.iloc[:, 1:-1]

In [33]:
train_data

,Brand,Variety,Style,Country,Stars
0,New Touch,T's Restaurant Tantanmen,Cup,Japan,3.75
1,Just Way,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,Pack,Taiwan,1
2,Nissin,Cup Noodles Chicken Vegetable,Cup,USA,2.25
3,Wei Lih,GGE Ramen Snack Tomato Flavor,Pack,Taiwan,2.75
4,Ching's Secret,Singapore Curry,Pack,India,3.75
...,...,...,...,...,...
2575,Vifon,"Hu Tiu Nam Vang [""Phnom Penh"" style] Asian Sty...",Bowl,Vietnam,3.5
2576,Wai Wai,Oriental Style Instant Noodles,Pack,Thailand,1
2577,Wai Wai,Tom Yum Shrimp,Pack,Thailand,2
2578,Wai Wai,Tom Yum Chili Flavor,Pack,Thailand,2


In [34]:
train_data["Variety"] = train_data["Variety"] + " " + train_data["Brand"] + " " + train_data["Style"] + " " + train_data["Country"]

train_data["Variety"] = train_data["Variety"].astype(str)

In [35]:
#Processing the text
stemmer = LancasterStemmer()

def processing(text):
  text = re.sub(r'\@\S+','',text)
  text = re.sub(r'http\:\/\/\S+','',text)
  text = re.sub(r'[^\w\s]','',text)
  tokens = text.lower().split(" ")
  stemmed_tokens = [stemmer.stem(x) for x in tokens]
  return ' '.join(stemmed_tokens)

train_data["Variety"] = train_data["Variety"].apply(processing)
train_data

,Brand,Variety,Style,Country,Stars
0,New Touch,ts resta tantanm new touch cup jap,Cup,Japan,3.75
1,Just Way,noodl spicy hot sesam spicy hot sesam guanmiao...,Pack,Taiwan,1
2,Nissin,cup noodl chick veget nissin cup us,Cup,USA,2.25
3,Wei Lih,gge ram snack tomato flav wei lih pack taiw,Pack,Taiwan,2.75
4,Ching's Secret,singap curry ching secret pack ind,Pack,India,3.75
...,...,...,...,...,...
2575,Vifon,hu tiu nam vang phnom penh styl as styl instan...,Bowl,Vietnam,3.5
2576,Wai Wai,ory styl inst noodl wai wai pack thailand,Pack,Thailand,1
2577,Wai Wai,tom yum shrimp wai wai pack thailand,Pack,Thailand,2
2578,Wai Wai,tom yum chil flav wai wai pack thailand,Pack,Thailand,2


In [38]:
vectorizer = CountVectorizer(binary=True,min_df=5)

#Fitting and processing countVectorizer
vectorizer.fit(train_data["Variety"])
train_vector = vectorizer.transform(train_data["Variety"])

In [24]:
#train_data_ohe = pd.get_dummies(train_data)
#train_data_ohe = train_data_ohe.fillna(0)

In [39]:
#K-means clustering
kmeans_v = KMeans(n_clusters=10, random_state=0).fit(train_vector)

In [40]:
kmeans_v.labels_

array([2, 4, 1, ..., 9, 9, 9], dtype=int32)

In [41]:
x = pd.read_csv("ramen-ratings.csv")
x = x.iloc[:, 1:-2]
x["label2"] = kmeans_v.labels_
x

,Brand,Variety,Style,Country,label2
0,New Touch,T's Restaurant Tantanmen,Cup,Japan,2
1,Just Way,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,Pack,Taiwan,4
2,Nissin,Cup Noodles Chicken Vegetable,Cup,USA,1
3,Wei Lih,GGE Ramen Snack Tomato Flavor,Pack,Taiwan,4
4,Ching's Secret,Singapore Curry,Pack,India,9
...,...,...,...,...,...
2575,Vifon,"Hu Tiu Nam Vang [""Phnom Penh"" style] Asian Sty...",Bowl,Vietnam,0
2576,Wai Wai,Oriental Style Instant Noodles,Pack,Thailand,3
2577,Wai Wai,Tom Yum Shrimp,Pack,Thailand,9
2578,Wai Wai,Tom Yum Chili Flavor,Pack,Thailand,9


In [31]:
#train_data_ohe["name"] = "sample0_" + x.index.astype(str) 
#train_data_ohe = train_data_ohe[ ["name"] + [ col for col in train_data_ohe.columns if col != "name"] ]
#train_data_ohe